# Computing $\pi$

In [4]:
import random
import math 
random.seed(3)

### 1. For loop 

In [11]:

def sample_point():
    "Sample point on [-1, 1] x [-1, 1]"
    x = random.uniform(-1, 1)
    y = random.uniform(-1, 1)
    return (x, y)


def in_radius(point):
    "Check whether point is in radius"
    x, y = point 
    dist = math.sqrt(x**2 + y**2)
    if dist <= 1:
        return 1 
    else:
        return 0


In [20]:
point = sample_point()
display(point)
in_radius(point)

(-0.39746468096857535, -0.9379764970605)

0

In [21]:
def calc_pi(N):
    """Computes the value of pi using N random samples."""
    n_points_in_radius = 0
    for i in range(N):
        point = sample_point()
        n_points_in_radius += in_radius(point)

    pi = 4 * n_points_in_radius / N
    return pi 

In [26]:
calc_pi(10_000_000)

3.1414028

In [32]:
%timeit calc_pi(10**7)

6.35 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2. Numpy 

In [30]:
import numpy as np

def calc_pi_numpy(N):
    # Simulate impact coordinates
    pts = np.random.uniform(-1, 1, (2, N))
    # Count number of impacts inside the circle
    M = np.count_nonzero((pts**2).sum(axis=0) < 1)
    return 4 * M / N

In [38]:
calc_pi_numpy(10000000)

3.1410728

In [33]:
%timeit calc_pi_numpy(10**7)

167 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3. Dask

In [34]:
import dask.array as da 

In [39]:
def calc_pi_dask(N):
    pts = da.random.uniform(-1, 1, (2, N))
    M = da.count_nonzero((pts**2).sum(axis=0) < 1)
    return 4 * M / N 

In [42]:
work = calc_pi_dask(100000)
work.compute()

3.14156

In [43]:
work = calc_pi_dask(10**7)
%timeit work.compute()

182 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3. Numba

In [47]:
import numba 

@numba.jit
def calc_pi_numba(N):
    M = 0
    for i in range(N):
        # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N

Note:
- I first tried to use the composed function from `calc_pi`, but without jit-ing the two inner functions `sample_point` and `in_radius`. 
- This threw an error. I assume I would need to use jit for all of the functions 

In [48]:
calc_pi_numba(100)

3.16

In [49]:
%timeit calc_pi_numba(10**7)

63.1 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
